# Kor LLM 모델 서빙 (SageMaker Python SDK)

### 참조: 
- Model 정보
    - beomi/KoAlpaca-Polyglot-12.8B
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KoAlpaca Dataset v1.1b
        - https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B
    - EleutherAI/polyglot-ko-12.8b
        - Polyglot-Ko-12.8B was trained for 167 billion tokens over 301,000 steps on 256 A100 GPUs with the GPT-NeoX framework. It was trained as an autoregressive language model, using cross-entropy loss to maximize the likelihood of predicting the next token.
        - License: Apache 2.0
        - https://huggingface.co/EleutherAI/polyglot-ko-12.8b
        
- Doc
    - Large model inference tutorials
        - https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-tutorials.html
    - Use DJL with the SageMaker Python SDK
        - https://sagemaker.readthedocs.io/en/stable/frameworks/djl/using_djl.html
        
- 블로그
    - https://aws.amazon.com/ko/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/
    - 코드
        - https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb

# 1. 기본 환경 설정

In [1]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../common_code')

# 2. SageMaker endpoint 의 추론 도커 이미지 인 DLC image URL 가져오기
- We get DLC image URL for djl-deepspeed 0.21.0 and set SageMaker settings

In [2]:
import sagemaker, boto3
from sagemaker import image_uris


role = sagemaker.get_execution_role()  # execution role for the endpoint
session = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = session._region_name
bucket = session.default_bucket()  # bucket to house artifacts

img_uri = image_uris.retrieve(framework="djl-deepspeed", region=region, version="0.21.0")
img_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117'

# 3. Set configuration

## 테스트 모델 지정

In [3]:
serve_model = 'KoAlpaca-12-8B'
# serve_model = 'Polyglot-Kor-5-8B'

# 모델 패키징 할 파일 이름
model_artifact_name = f'{serve_model}.tar.gz'

# 모델 패키징 S3 위치
s3_location = f"s3://{bucket}/{serve_model}"

print("serve_model: ", serve_model)
print("model_artifact_name: ", model_artifact_name)
print("model packaging s3 location: ", s3_location)

serve_model:  KoAlpaca-12-8B
model_artifact_name:  KoAlpaca-12-8B.tar.gz
model packaging s3 location:  s3://sagemaker-us-east-1-057716757052/KoAlpaca-12-8B


## 모델 다운로드

In [4]:
##수정중
# from huggingface_hub import snapshot_download
# from pathlib import Path
# import os

# # - This will download the model into the current directory where ever the jupyter notebook is running
# local_model_path = Path(".")
# local_model_path.mkdir(exist_ok=True)
# model_name = "beomi/KoAlpaca-Polyglot-12.8B"

# # Only download pytorch checkpoint files
# allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# # - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
# model_download_path = snapshot_download(
#     repo_id=model_name,
#     cache_dir=local_model_path,
#     allow_patterns=allow_patterns,
# )
# model_download_path

## 로컬 모드 혹은 클라우드 모드

In [5]:
use_local_mode = False
# use_local_mode = True

if use_local_mode:
    instance_type = "local_gpu"
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()
    instance_type = "ml.g5.12xlarge"
    
print("instance_type  :", instance_type)      

instance_type  : ml.g5.12xlarge


# 4. 모델 추론 코드 및 모델 설정 파일을 패키징
- `model.py` and `serving.properties`
- The code below creates the SageMaker model file (`model.tar.gz`) and upload it to S3. 

In [6]:
%%sh -s {serve_model} {model_artifact_name}
serve_model=$1
model_artifact_name=$2
echo $serve_model
echo $model_artifact_name

rm -rf $serve_model/.ipynb_checkpoints

tar -czvf $model_artifact_name $serve_model/

KoAlpaca-12-8B
KoAlpaca-12-8B.tar.gz
KoAlpaca-12-8B/
KoAlpaca-12-8B/serving.properties
KoAlpaca-12-8B/model.py


## mode.tar.gz 및 pretrained model을 S3 업로드

In [7]:
model_tar_url = sagemaker.s3.S3Uploader.upload(model_artifact_name, s3_location)
print("model_tar_url: ", model_tar_url)

model_tar_url:  s3://sagemaker-us-east-1-057716757052/KoAlpaca-12-8B/KoAlpaca-12-8B.tar.gz


In [8]:
## 수정 중
# define a variable to contain the s3url of the location that has the model
#pretrained_model_location = f'{s3_location}/pretrained/' 
#print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")

In [9]:
## 수정 중
# model_artifact = session.upload_data(path=model_download_path, key_prefix=f'{serve_model}/pretrained')
# print(f"Model uploaded to --- > {model_artifact}")
# print(f"We will set option.s3url={model_artifact}")

In [10]:
## 수정 중
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
#jinja_env = jinja2.Environment()
#template = jinja_env.from_string(Path("mymodel/serving.properties").open().read())
#Path("mymodel/serving.properties").open("w").write(template.render(s3url=pretrained_model_location))
#!pygmentize mymodel/serving.properties | cat -n

# 5. SageMaker Model 생성

In [11]:
from sagemaker.model import Model
from sagemaker import image_uris, get_execution_role

def create_model(model_name, role, sagemaker_session, inference_image_uri, model_s3_url):
    model = Model(
        image_uri=inference_image_uri,
        model_data=model_s3_url,
        role=role,
        name=model_name,
        sagemaker_session=sagemaker_session,
    )
    return model

In [12]:
from datetime import datetime

time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model_name = f"{serve_model}-" + time_stamp

sm_model = create_model(model_name, role, sagemaker_session, img_uri, model_tar_url)

# 6. SageMaker Endpoint 생성
- 클라우드 배포시 약 8분 걸림. 10 이상 걸리면 무엇인가 문제 있음

In [13]:
from sagemaker import serializers, deserializers

def deploy_model(model, sagemaker_session, instance_type, _endpoint_name):
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=_endpoint_name
    )
    predictor = sagemaker.Predictor(
        endpoint_name=_endpoint_name,
        sagemaker_session=sagemaker_session,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer()
    )
    return predictor

In [14]:
%%time
endpoint_name = f"{serve_model}-" + time_stamp
predictor = deploy_model(sm_model, sagemaker_session, instance_type, endpoint_name)

------------------------!CPU times: user 127 ms, sys: 41.2 ms, total: 169 ms
Wall time: 12min 34s


# 7. 엔드포인트 추론 

In [17]:
from inference_lib import invoke_inference_DJ    

### options for generation
* **temperature**: Controls randomness in the model. Lower values will make the model more deterministic and higher values will make the model more random. Default value is 1.0.
* **max_new_tokens**: The maximum number of tokens to generate. Default value is 20, max value is 512.
* **repetition_penalty**: Controls the likelihood of repetition, defaults to null.
* **seed**: The seed to use for random generation, default is null.
* **stop**: A list of tokens to stop the generation. The generation will stop when one of the tokens is generated.
* **top_k**: The number of highest probability vocabulary tokens to keep for top-k-filtering. Default value is null, which disables top-k-filtering.
* **top_p**: The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling, default to null
* **do_sample**: Whether or not to use sampling ; use greedy decoding otherwise. Default value is false.
* **best_of**: Generate best_of sequences and return the one if the highest token logprobs, default to null.
* **details**: Whether or not to return details about the generation. Default value is false.
* **return_full_text**: Whether or not to return the full text or only the generated part. Default value is false.
* **truncate**: Whether or not to truncate the input to the maximum length of the model. Default value is true.
* **typical_p**: The typical probability of a token. Default value is null.
* **watermark**: The watermark to use for the generation. Default value is false.

In [18]:
params = {
    "do_sample":False, 
    "max_new_tokens":128,
    "temperature":1.0,
    "top_k":0,
    "top_p":0.9,
    "return_full_text":False,
    "repetition_penalty":1.1,
    "presence_penalty":None,
    "eos_token_id":2,
}

## (1) 맥락 (Context) 없이 질문

In [19]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = ""#"홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다."
prompt_wo_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
data = {
    "prompt": [prompt_wo_c,],
    "params": params
}
print("prompt_wo_c: \n", data)

prompt_wo_c: 
 {'prompt': ['### 질문: 홈플러스 중계점은 몇시까지 장사해?\n\n### 답변:'], 'params': {'do_sample': False, 'max_new_tokens': 128, 'temperature': 1.0, 'top_k': 0, 'top_p': 0.9, 'return_full_text': False, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}}


In [20]:
%%time 
invoke_inference_DJ(endpoint_name, data)

[
  [
    {
      "generated_text":"홈플러스의 매장 운영 시간은 점포마다 다릅니다. 대부분의 매장은 10시에 문을 닫으며, 일부 매장은 11시까지 영업합니다. 예를 들어, 홈플러스 동대문점은 11시까지 영업하며, 홈플러스 영등포점과 강서점은 9시에 문을 닫습니다. 또한, 홈플러스 김해점과 밀양점은 8시에 문을 닫고 있습니다. "
    }
  ]
]
CPU times: user 52.3 ms, sys: 10.6 ms, total: 63 ms
Wall time: 5.79 s


'[\n  [\n    {\n      "generated_text":"홈플러스의 매장 운영 시간은 점포마다 다릅니다. 대부분의 매장은 10시에 문을 닫으며, 일부 매장은 11시까지 영업합니다. 예를 들어, 홈플러스 동대문점은 11시까지 영업하며, 홈플러스 영등포점과 강서점은 9시에 문을 닫습니다. 또한, 홈플러스 김해점과 밀양점은 8시에 문을 닫고 있습니다. "\n    }\n  ]\n]'

## (2) 맥락 (Context) 가지고 질문

In [21]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = "홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 "
prompt_w_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
data = {
    "prompt": [prompt_w_c,],
    "params": params
}
print("prompt_w_c:\n", prompt_w_c)

prompt_w_c:
 ### 질문: 홈플러스 중계점은 몇시까지 장사해?

### 맥락: 홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 

### 답변:


In [22]:
%%time 
invoke_inference_DJ(endpoint_name,  data)

[
  [
    {
      "generated_text":"홈플러스의 영업 시간은 오전 10시부터 오후 10시까지입니다. 각 지점마다 약간씩 차이가 있을 수 있으므로, 방문 전에 영업 시간을 꼭 확인해보시기 바랍니다. 아래 링크에서 홈플러스 매장 찾기를 통해 영업 시간을 확인하실 수 있습니다: http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%"
    }
  ]
]
CPU times: user 15.3 ms, sys: 1.32 ms, total: 16.6 ms
Wall time: 8.3 s


'[\n  [\n    {\n      "generated_text":"홈플러스의 영업 시간은 오전 10시부터 오후 10시까지입니다. 각 지점마다 약간씩 차이가 있을 수 있으므로, 방문 전에 영업 시간을 꼭 확인해보시기 바랍니다. 아래 링크에서 홈플러스 매장 찾기를 통해 영업 시간을 확인하실 수 있습니다: http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%"\n    }\n  ]\n]'

# 7. [중요] 클린업 엔트포인트 

In [23]:
# delete endpoint
predictor.delete_model()
predictor.delete_endpoint()

# Trouble Shooting

3vep2qi5ar-algo-1-zk8i8 | INFO  ModelServer Model server stopped.

3vep2qi5ar-algo-1-zk8i8 | ERROR ModelServer Invalid configuration: Workflow KoAlpaca_12_8B is already registered.